# CMIP6Plus MIP table remapping
To support the extension of CMIP6Plus variable set the CMIP6 variables have been reallocated to a set of tables named by their modelling domain, frequency and variable shape,
e.g. atmos physical variables have moved from MIP table `Amon` to `APmon`, while atmospheric chemistry has moved to `ACmon` and aerosols to `AEmon`. Where data is on model levels `Lev` is appended to the table name.

The code below constructs a dictionary to map the name of the CMIP6 variable to the CMIP6Plus equivalent

In [1]:
import glob
import json
import os

Load table files and ignore coordinate and generic CV information

In [2]:

file_name_endings_to_ignore = [
    'coordinate.json',
    'formula_terms.json',
    'CV.json',
    'grids.json']

table_files = []
for i in glob.glob('../Tables/*'):
    if not any(i.endswith(j) for j in file_name_endings_to_ignore):
        table_files.append(i)
    else:
        print('Ignoring ancillary file "{}"'.format(i))
        
print('Found {} files'.format(len(table_files)))

Ignoring ancillary file "../Tables/MIP_formula_terms.json"
Ignoring ancillary file "../Tables/MIP_coordinate.json"
Ignoring ancillary file "../Tables/MIP_grids.json"
Ignoring ancillary file "../Tables/CMIP6Plus_CV.json"
Found 77 files


Loop over variables and extract the CMIP6 variable name from the mip table entry and add to a dictionary

In [3]:
cmip6_to_cmip6plus_variable_mapping = {}
for table_filename in table_files:
    table = json.load(open(table_filename))
    table_id = table['Header']['table_id']
    for varname, details in table['variable_entry'].items():
        if 'CMIP6' in details['provenance']:
            cmip6_table_id = details['provenance']['CMIP6']['mip_table']
            cmip6_varname = details['provenance']['CMIP6']['variable_name']
            cmip6_to_cmip6plus_variable_mapping[(cmip6_table_id, cmip6_varname)] = (table_id, varname)

Print the result

In [4]:
print('(CMIP6 MIP table, CMIP6 Variable name) : (CMIP6Plus MIP table, CMIP6Plus Variable name)')

for cmip6_var in sorted(cmip6_to_cmip6plus_variable_mapping):
    print(cmip6_var, ':', cmip6_to_cmip6plus_variable_mapping[cmip6_var])

(CMIP6 MIP table, CMIP6 Variable name) : (CMIP6Plus MIP table, CMIP6Plus Variable name)
('3hr', 'clt') : ('AP3hr', 'clt')
('3hr', 'hfls') : ('AP3hr', 'hfls')
('3hr', 'hfss') : ('AP3hr', 'hfss')
('3hr', 'huss') : ('AP3hrPt', 'huss')
('3hr', 'mrro') : ('LP3hr', 'mrro')
('3hr', 'mrsos') : ('LP3hrPt', 'mrsos')
('3hr', 'pr') : ('AP3hr', 'pr')
('3hr', 'prc') : ('AP3hr', 'prc')
('3hr', 'prsn') : ('AP3hr', 'prsn')
('3hr', 'ps') : ('AP3hrPt', 'ps')
('3hr', 'rlds') : ('AP3hr', 'rlds')
('3hr', 'rldscs') : ('AP3hr', 'rldscs')
('3hr', 'rlus') : ('AP3hr', 'rlus')
('3hr', 'rsds') : ('AP3hr', 'rsds')
('3hr', 'rsdscs') : ('AP3hr', 'rsdscs')
('3hr', 'rsdsdiff') : ('AP3hr', 'rsdsdiff')
('3hr', 'rsus') : ('AP3hr', 'rsus')
('3hr', 'rsuscs') : ('AP3hr', 'rsuscs')
('3hr', 'tas') : ('AP3hrPt', 'tas')
('3hr', 'tos') : ('OP3hrPt', 'tos')
('3hr', 'tslsi') : ('LP3hrPt', 'tslsi')
('3hr', 'uas') : ('AP3hrPt', 'uas')
('3hr', 'vas') : ('AP3hrPt', 'vas')
('6hrLev', 'bs550aer') : ('AE6hrPtLev', 'bs550aer')
('6hrLev', '

Determine the mapping between tables in the two projects; There will be some tables where there is not a 1:1 mapping

In [5]:

from collections import defaultdict
table_mapping = defaultdict(list)
reverse_table_mapping = defaultdict(list)
for table_filename in table_files:
    table = json.load(open(table_filename))
    table_id = table['Header']['table_id']
    # table_mapping[table_id] = defaultdict(list)
    for varname, details in table['variable_entry'].items():
        if 'CMIP6' in details['provenance']:
            cmip6_table_id = details['provenance']['CMIP6']['mip_table']
            if table_id not in table_mapping[cmip6_table_id]:
                table_mapping[cmip6_table_id].append(table_id)
            if cmip6_table_id not in reverse_table_mapping[table_id]:
                reverse_table_mapping[table_id].append(cmip6_table_id)

# cmip6+ : cmip6
dict(reverse_table_mapping)

{'ACmon': ['Emon'],
 'ACmonZ': ['EmonZ'],
 'AE1hr': ['AERhr'],
 'AE3hrPt': ['E3hrPt'],
 'AE3hrPtLev': ['E3hrPt'],
 'AE6hr': ['6hrPlev'],
 'AE6hrPt': ['6hrPlevPt'],
 'AE6hrPtLev': ['6hrLev'],
 'AEday': ['AERday'],
 'AEmon': ['AERmon', 'Emon'],
 'AEmonLev': ['AERmon', 'Emon'],
 'AEmonZ': ['AERmonZ', 'EmonZ'],
 'AEsubhrPt': ['Esubhr'],
 'AEsubhrPtSite': ['Esubhr'],
 'AP1hr': ['E1hr', 'AERhr'],
 'AP1hrPt': ['E1hr'],
 'AP3hr': ['E3hr', '3hr'],
 'AP3hrPt': ['E3hrPt', 'CF3hr', '3hr'],
 'AP3hrPtLev': ['E3hrPt', 'CF3hr'],
 'AP6hr': ['6hrPlev'],
 'AP6hrPt': ['6hrPlevPt', '6hrLev'],
 'AP6hrPtLev': ['6hrLev'],
 'AP6hrPtZ': ['E6hrZ'],
 'APday': ['CFday', 'Eday', 'day', 'AERday'],
 'APdayLev': ['CFday'],
 'APdayZ': ['EdayZ'],
 'APfx': ['fx', 'Efx'],
 'APmon': ['CFmon', 'Amon', 'Emon', 'Omon'],
 'APmonClim': ['Amon'],
 'APmonClimLev': ['Amon'],
 'APmonDiurnal': ['E1hrClimMon'],
 'APmonLev': ['Amon', 'CFmon', 'Emon', 'AERmon'],
 'APmonZ': ['EmonZ', 'AERmonZ'],
 'APsubhrPt': ['Esubhr'],
 'APsubhrPtLev'

In [6]:
json.dump(table_mapping, open('table_6_to_6plus.json','w'),indent=2)
json.dump(reverse_table_mapping, open('table_6plus_to_6.json','w'),indent=2)